# SEP532 인공지능 이론과 실제
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

## Information
**Please write your student ID and name here!**
- Student ID: 20223751
- Name: 강병욱

---

## Assignment: Action Recognition

In this assignment, you are going to recognize an action of a given video using both a convolutional neural network and a recurrent neural network. Please follow the steps below to continue this assignment.

### Guidelines
Copy this example to your Google Colab and edit it to complete your assignment. We should be able to reproduce your results using your code and pre-trained model. Please double-check if your code runs without error properly. Submissions failed to run or reproduce the results will get a substantial penalty.

In [1]:
import os
import random
import re

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

# DO NOT EDIT THE FOLLOWING LINES
# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

### 1. Load the UCF101 dataset

In this assignment, you will use the UCF101 which is an action recognition dataset of realistic action videos, collected from YouTube, having 101 action categories. (*Soomro, K., Zamir, A. R., & Shah, M. (2012). UCF101: A dataset of 101 human actions classes from videos in the wild. arXiv preprint arXiv:1212.0402.*)

![UCF101 Dataset](images/ucf101.jpg)

The UCF101 dataset consists of 13,320 videos and their labels. Since your computing resource in Google Colab is somewhat limited, TA sampled half of the dataset, limited the length of videos to 64 frames, separated videos into frames and stored them to a single file in advance. 

Let's download and load this file. **(If the link below is not working, please let TA know to fix the link)**

In [2]:
import pickle

if not os.path.exists('ucf101.pickle'):
    !wget -O 'ucf101.pickle' 'https://www.dropbox.com/s/2558ailo46px55j/ucf101.pickle?dl=1'

    # If the link above is not working, you can also use the following link but it would be slower than the above.
    # !wget -O 'ucf101.pickle' 'https://storage.keai.io/s/0PdikjRn2xFl2ks/download'
    
with open('ucf101.pickle', 'rb') as input_file:
    dataset = pickle.load(input_file)
    
num_trains = len(dataset['train'])
num_validations = len(dataset['validation'])
num_tests = len(dataset['test'])

In [3]:
index_to_label = [
    'ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 
    'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 
    'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 
    'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 
    'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 
    'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 
    'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 
    'HammerThrow', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 
    'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 
    'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 
    'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 
    'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 
    'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 
    'PlayingTabla', 'PlayingViolin', 'PoleVault', 'PommelHorse', 'PullUps', 
    'Punch', 'PushUps', 'Rafting', 'RockClimbingIndoor', 'RopeClimbing', 
    'Rowing', 'SalsaSpin', 'ShavingBeard', 'Shotput', 'SkateBoarding', 
    'Skiing', 'Skijet', 'SkyDiving', 'SoccerJuggling', 'SoccerPenalty', 
    'StillRings', 'SumoWrestling', 'Surfing', 'Swing', 'TableTennisShot', 
    'TaiChi', 'TennisSwing', 'ThrowDiscus', 'TrampolineJumping', 'Typing', 
    'UnevenBars', 'VolleyballSpiking', 'WalkingWithDog', 'WallPushups', 'WritingOnBoard', 
    'YoYo', 
]

Let's visualize what some of these videos and their corresponding labels look like.

In [4]:
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

def plot_frames(frames):
    figure = plt.figure(figsize=(frames.shape[1] / 72, frames.shape[2] / 72), dpi=72)
    image = plt.figimage(frames[0])
    
    def animate(step):
        image.set_array(frames[step])
        return (image, )
    
    video = FuncAnimation(
        figure, animate, 
        frames=len(frames), interval=33, 
        repeat_delay=1, repeat=True
    ).to_html5_video()
    
    display(HTML(video))

In [5]:
for frames, label in random.sample(dataset['train'], 3):
    plot_frames(tf.stack([tf.image.decode_jpeg(frame) for frame in frames]))
    print('Label:', index_to_label[label])

Label: PlayingDaf


Label: HammerThrow


Label: WalkingWithDog


<Figure size 256x256 with 0 Axes>

<Figure size 256x256 with 0 Axes>

<Figure size 256x256 with 0 Axes>

### 2. Preprocess the dataset

Unlike images and text, video data contains both spatial and temporal information. Therefore, to handle these data, you will use both convolutional neural networks and recurrent neural networks to recognize an action of the videos.

First, let's extract meaningful features from video frames using the pre-trained convolutional neural networks.

#### Problem 1
Define a model to extract meaningful features from the given video frame using the pre-trained convolutional neural networks of your choice.

This model should output a 1D vector for one given frame. That is,
- In: `(1, 256, 256, 3)` → Out: `(1, dimension of features of your choice)`
- In: `(5, 256, 256, 3)` → Out: `(5, dimension of features of your choice)`
- In: `(number of frames, 256, 256, 3)` → Out: `(number of frames, dimension of features of your choice)`

In [6]:
# TODO: Define a model to extract features from the given video frame
#       using the pre-trained convolutional neural networks of your choice.

### START CODE HERE ###
cnn_model = tf.keras.applications.InceptionV3(input_shape=(256,256,3),
                                               include_top=False,
                                                weights='imagenet',
                                                 pooling='avg')

cnn_model.trainable = False
cnn_model.summary()
### END CODE HERE ###

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

#### Problem 2
To extract features from the given frames, TA provided `extract_features()` function. In this function, you need to **preprocess each frame** so it can be fed into the `cnn_model`

In [7]:
# This function extracts features from the given frames using the defined cnn_model
# - In: frames.shape = (number of frames, 256, 256, 3)
# - Out: features.shape = (number of frames, dimension of features of your choice)
def extract_features(frames, batch=32):
    # TODO: Preprocess each frame so it can be fed into the `cnn_model`
    ### START CODE HERE ###
    frames = tf.cast(frames, tf.float32)
    frames = (frames/127.5) - tf.constant(1, dtype=tf.float32)
    ### END CODE HERE ###
    
    features = tf.concat([
        cnn_model(frames[index:index + batch])
        for index in range(0, frames.shape[0], batch)
    ], axis=0)
    
    if features.shape[0] < max_length:
        features = tf.concat([
            features,
            tf.zeros((max_length - features.shape[0], *features.shape[1:]))
        ], axis=0)
    
    return features.numpy()

Now, you can extract features from the video but this task is very time-consuming. Therefore, TA provided `preprocess_dataset()` function which takes a dataset, extracts features from the dataset, stores the features into a file, and loads the features from the file.

In [8]:
import os
import pickle

max_length = 32 # DO NOT CHANGE THIS NUMBER

def decode_frames(frames):
    return tf.stack([tf.image.decode_jpeg(frame) for frame in frames])

def preprocess_dataset(dataset, filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as input_file:
            return pickle.load(input_file)
    else:
        tensors = [
            (extract_features(decode_frames(frames[:max_length])), label)
            for frames, label in tqdm(dataset)
        ]
        
        X, y = zip(*tensors)
        X, y = np.array(X), np.array(y)
        
        with open(filename, 'wb') as output_file:
            pickle.dump((X, y), output_file, protocol=4) # protocol=4 supports a byte objects larget than 4GB
        
        return (X, y)

Let's extract features from all the datasets. **Please note that the first run takes up to 30 minutes.**

In [9]:
train_features, train_labels = preprocess_dataset(dataset['train'], f'train-dataset-{max_length}-{num_trains}.pickle')
validation_features, validation_labels = preprocess_dataset(dataset['validation'], f'validation-dataset-{max_length}-{num_validations}.pickle')
test_features, test_labels = preprocess_dataset(dataset['test'], f'test-dataset-{max_length}-{num_tests}.pickle')

Then, combine the features into batches.

In [10]:
batch_size = 64

batch_train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).shuffle(512).batch(batch_size)
batch_validation_dataset = tf.data.Dataset.from_tensor_slices((validation_features, validation_labels)).shuffle(512).batch(batch_size)
batch_test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).shuffle(512).batch(batch_size)

### 3. Build the model
All videos are transformed into 2D tensors via convolutional neural networks. To process these tensors, let's build a recurrent neural network.

#### Problem 3
Define a recurrent neural network to recognize one of `num_classes` actions from the given video. Becaue all videos have different lengths, your `lstm_model` should take account this into account. To do that, TA added `tf.keras.layers.Masking` layer in advance.

In [11]:
from tensorflow.python.ops.nn_ops import dropout
num_classes = len(index_to_label)

lstm_model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.0), # DO NOT REMOVE THIS LAYER

    # TODO: Define a recurrent neural network to recognize one of `num_classes` actions from the given video
    ### START CODE HERE ###
    tf.keras.layers.GRU(1024, return_sequences=False),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax")
    ### END CODE HERE ###
])

### 4. Train the model
In this assignment, you are going to manually train your model using `tf.GradientTape`.

#### Problem 4
Define an optimizer of your choice and optimize your `lstm_model` to minimize `loss`

In [12]:
# TODO: Define an optimizer of your choice
### START CODE HERE ###
optimizer = tf.keras.optimizers.Adam()
### END CODE HERE ###

def forward(features, actual):
    predicted = lstm_model(features)
    loss = tf.keras.losses.sparse_categorical_crossentropy(actual, predicted)

    return predicted, tf.reduce_mean(loss)

def train_step(features, actual):
    with tf.GradientTape() as tape:
        predicted, loss = forward(features, actual)
    
    # TODO: Optimize your `lstm_model` to minimize `loss`
    ### START CODE HERE ###
    gradients = tape.gradient(loss, lstm_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, lstm_model.trainable_variables))
    ### END CODE HERE ###
    
    return predicted, loss

In [13]:
def evaluate(batch_dataset):
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    test_loss = tf.keras.metrics.Mean()
    
    for batch_features, batch_actual in batch_dataset:
        batch_predicted, batch_loss = forward(batch_features, batch_actual)
        
        test_accuracy(batch_actual, batch_predicted)
        test_loss(batch_loss)
        
    return test_accuracy.result(), test_loss.result()

#### Problem 5
Train your `lstm_model` at least 10 epochs. *Please note that if you want to use your custom training loop, it is completely okay to use your loop instead of the code cell below.*

In [14]:
epochs = 30 # YOU CAN ADJUST THE NUMBER OF EPOCHS IF YOU WANT

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
train_loss = tf.keras.metrics.Mean()

with tqdm(total=epochs) as epoch_progress:
    for epoch in range(epochs):
        epoch_progress.set_description(f'Epoch {epoch + 1}')
        
        train_accuracy.reset_states()
        train_loss.reset_states()
        
        with tqdm(total=num_trains // batch_size) as batch_progress:
            for batch, (batch_features, batch_actual) in enumerate(batch_train_dataset):
                batch_progress.set_description(f'Batch {batch + 1}')
                
                batch_predicted, batch_loss = train_step(batch_features, batch_actual)
                train_accuracy(batch_actual, batch_predicted)
                train_loss(batch_loss)
                
                if (batch % 10) == 0:
                    batch_progress.set_postfix(batch_loss=batch_loss.numpy())
                batch_progress.update()
        
            validation_accuracy, validation_loss = evaluate(batch_validation_dataset)
            batch_progress.set_postfix(
                batch_loss=batch_loss.numpy(),
                validation_accuracy=validation_accuracy.numpy(), 
                validation_loss=validation_loss.numpy())
        
        epoch_progress.set_postfix(
            train_accuracy=train_accuracy.result().numpy(), 
            train_loss=train_loss.result().numpy(),
            validation_accuracy=validation_accuracy.numpy(), 
            validation_loss=validation_loss.numpy())
        epoch_progress.update()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

...

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

### 4. Evaluate accuracy

Let's evaluate the trained model using test dataset and print the test accuracy of the model. For your information, the accuracy of the model proposed by the authors who published the UCF101 dataset is 43.90%.

In [15]:
test_accuracy, test_loss = evaluate(batch_test_dataset)

print(f'Test accuracy: {test_accuracy.numpy():.4f}')
print(f'Test loss: {test_loss.numpy():.4f}')

Test accuracy: 0.6536
Test loss: 1.8057


Using the below cell, you can try to recognize an action of the test videos using your trined `lstm_model`.

In [16]:
for frames, label in random.sample(dataset['test'], 3):
    print('Acutal:', index_to_label[label])
    
    print('Predicted:')
    features = extract_features(decode_frames(frames[:max_length]))
    predicted = lstm_model(tf.expand_dims(features, 0))[0]
    for confidence, index in zip(*tf.math.top_k(predicted, k=3)):
        print(f'- {index_to_label[index]} ({confidence.numpy():.4f})')
    
    plot_frames(decode_frames(frames))
    print()

Acutal: BoxingPunchingBag
Predicted:
- BoxingPunchingBag (0.9616)
- PizzaTossing (0.0243)
- BoxingSpeedBag (0.0069)



Acutal: HorseRace
Predicted:
- HorseRace (0.9978)
- SoccerPenalty (0.0017)
- CricketBowling (0.0004)



Acutal: BalanceBeam
Predicted:
- FloorGymnastics (0.9847)
- Diving (0.0146)
- CleanAndJerk (0.0001)


<Figure size 256x256 with 0 Axes>

<Figure size 256x256 with 0 Axes>

<Figure size 256x256 with 0 Axes>

#### Extra points
TAs will rank the submissions based on the test accuracy and assign extra points according to the rank.